In [6]:
import numpy as np
from PIL import Image
from pathlib import Path
import math

In [2]:
class ID(object):
	array = None
	def __init__(self, n = 4):
		self.array = []
		for i in range(n):
			self.array.append(0)

	def __iadd__(self, n):
		if math.log10(n) > len(self.array):
			raise ValueError("Overflow imminent. Halting.")
		i = len(self.array) - 1
		add = n
		while i >= 0:
			current = add%10
			self.array[i]+= current
			add = add//10
			if self.array[i] >= 10:
				self.array[i] = 0
				add+= 1
			i-= 1
		return self

	def __add__(self, n):
		new = ID()
		new.array = self.array.copy()
		new+= n
		return new

	def __str__(self):
		return "".join(map(str, self.array))

	def __repr__(self):
		return self.__str__()

In [3]:
images = Path("./Data/Images")
labels = Path("./Data/Labels")
npzs = Path("./Data/npzs")

if not npzs.exists():
	npzs.mkdir()

In [7]:
name = ID(4)
names = []
for image in images.iterdir():
	# print(image.name)
	img_file = Image.open(image)
	label_file = Image.open(labels / image.name)
	img = np.array(img_file)
	label = np.array(np.array(label_file)/255, dtype=int)
	# Conver the arrays from 64X64 to 224X224 by adding 0s
	# img = np.pad(img, ((80, 80), (80, 80)), 'constant')
	# label = np.array(np.pad(label, ((80, 80), (80, 80)), 'constant'), dtype=int)
	np.savez(npzs / f"image{name}", image=img, label=label)
	names.append(f"image{name}")
	name+= 1
file = open("names.txt", "w")
file.write("\n".join(names))
file.close()

In [ ]:
# Test if the npzs are correct
for npz in npzs.iterdir():
	npz_file = np.load(npz)
	print(npz_file["image"].shape, npz_file["label"].shape)